In [33]:
%pip install pyspark


     ---------------------------------------- 0.0/316.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/316.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/316.9 MB ? eta -:--:--
     -------------------------------------- 0.0/316.9 MB 217.9 kB/s eta 0:24:15
     -------------------------------------- 0.1/316.9 MB 357.2 kB/s eta 0:14:47
     -------------------------------------- 0.2/316.9 MB 981.9 kB/s eta 0:05:23
     ---------------------------------------- 0.4/316.9 MB 1.3 MB/s eta 0:03:55
     ---------------------------------------- 0.6/316.9 MB 1.8 MB/s eta 0:02:56
     ---------------------------------------- 0.8/316.9 MB 2.3 MB/s eta 0:02:17
     ---------------------------------------- 1.2/316.9 MB 2.8 MB/s eta 0:01:52
     ---------------------------------------- 1.4/316.9 MB 3.0 MB/s eta 0:01:46
     ---------------------------------------- 1.6/316.9 MB 3.3 MB/s eta 0:01:35
     ---------------------------------------- 1.9/316.9 MB 3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from pyspark.sql import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tracks = pd.read_csv('archive\SpotifyFeatures.csv')
tracks.head()

In [ ]:
# tracks.shape
# tracks.info

tracks.isnull().sum()

In [ ]:
tracks.dropna(inplace=True)
tracks.isnull().sum().plot.bar()
plt.show()

In [ ]:
tracks = tracks.drop(['track_id'],axis=1)

In [ ]:
trackstsne = tracks.drop(['track_id','genre','artist_name','track_name','key','mode','time_signature'],axis=1)
model = TSNE(n_components=2, random_state=0)
tsne_data = model.fit_transform(trackstsne.head(500))
plt.figure(figsize=(7,7))
plt.scatter(tsne_data[:,0],tsne_data[:,1])
plt.show()

In [ ]:
# tracks['track_name'].nunique(),tracks.shape

# tracks = tracks.sort_values(by=['popularity'],ascending=False)
# tracks.drop_duplicates(subset=['name'],keep='first',inplace=True)

# plt.figure(figsize=(10,5))
# sb.countplot(tracks[''])

In [ ]:
tracks = pd.read_csv('archive\Spotify.csv')

song_vectorizer = CountVectorizer()
song_vectorizer.fit(tracks['genre'])



def get_similarities(song_name, data):

# Getting vector for the input song.
    text_array1 = song_vectorizer.transform(data[data['track_name']==song_name]['genre']).toarray()
    num_array1 = data[data['track_name']==song_name].select_dtypes(include=np.number).to_numpy()

    # We will store similarity for each row of the dataset.
    sim = []
    for idx, row in data.iterrows():
        name = row['track_name']
        
        # Getting vector for current song.
        text_array2 = song_vectorizer.transform(data[data['track_name']==name]['genre']).toarray()
        num_array2 = data[data['track_name']==name].select_dtypes(include=np.number).to_numpy()

        # Calculating similarities for text as well as numeric features
        text_sim = cosine_similarity(text_array1, text_array2)[0][0]
        num_sim = cosine_similarity(num_array1, num_array2)[0][0]
        sim.append(text_sim + num_sim)
        
    return sim

def recommend_songs(song_name, data=tracks):
# Base case
    if tracks[tracks['track_name'] == song_name].shape[0] == 0:
        print('This song is either not so popular or you\
        have entered invalid_name.\n Some songs you may like:\n')
        
        for song in data.sample(n=5)['artist_name'].values:
            print(song)
        return

    data['similarity_factor'] = get_similarities(song_name, data)

    data.sort_values(by=['similarity_factor', 'popularity'],
                    ascending = [False, False],
                    inplace=True)

    # First song will be the input song itself as the similarity will be highest.
    display(data[['track_name', 'artist_name']][2:7])

recommend_songs('Le Fêtard')


